In [3]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
import numpy as np
import string
import os
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import xgboost

import tsfresh
from tsfresh import extract_features


In [4]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [5]:
import importlib


import board_helpers
from board_helpers.move_info import MoveInfo
from board_helpers.tiles_counter import TilesCoutner
from board_helpers.board_consts import TileType
importlib.reload(board_helpers)
importlib.reload(board_helpers.move_info)
importlib.reload(board_helpers.tiles_counter)
importlib.reload(board_helpers.board_consts)



import transformers
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer
from transformers.add_to_dict_transformer import AddToDictTransformer
from transformers.get_from_dict_transformer import GetFromDictTransformer
from transformers.select_pos_rows_transformer import SelectPosRowsTransformer
importlib.reload(transformers)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)
importlib.reload(transformers.add_to_dict_transformer)
importlib.reload(transformers.get_from_dict_transformer)
importlib.reload(transformers.select_pos_rows_transformer)

import functions
from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions)
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)

import processors
from processors.basic_pre_processor import BasicPreProcessor
importlib.reload(processors)
importlib.reload(processors.basic_pre_processor)

import builders
from builders.preprocessor_builder import PreprocessorBuilder
from builders.reg_pipe_builder import RegPipeBuilder
importlib.reload(builders)
importlib.reload(builders.preprocessor_builder)
importlib.reload(builders.reg_pipe_builder)

import searcher
import searcher_results_orgenizer
from searcher import Searcher
from searcher_results_orgenizer import SearcherResultsOrgenizer
importlib.reload(searcher)
importlib.reload(searcher_results_orgenizer)







<module 'searcher_results_orgenizer' from 'c:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\searcher_results_orgenizer.py'>

In [6]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/features_filtered.csv', index_col='game_id')

In [7]:
turns

,.__mean,score__absolute_maximum,.__sum_values,.__median,.__root_mean_square,points__root_mean_square,score__maximum,TileType.N__sum_values,points__median,points__sum_values,...,W__absolute_maximum,T__sum_values,B__absolute_maximum,B__maximum,C__maximum,C__absolute_maximum,H__maximum,H__absolute_maximum,A__median,turn_type_Pass__median
game_id,,,,,,,,,,,,,,,,,,,,,
1,0.444444,429.0,12.0,0.0,0.816497,35.211530,429.0,78.0,24.0,764.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
3,0.481481,440.0,13.0,0.0,0.793492,34.230158,440.0,74.0,25.0,758.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,1.137931,478.0,33.0,1.0,1.790781,28.373737,478.0,73.0,16.0,597.0,...,1.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0
5,0.451613,427.0,14.0,0.0,0.762001,29.444316,427.0,74.0,21.0,752.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
6,0.600000,427.0,15.0,0.0,1.000000,37.758708,427.0,78.0,28.0,805.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,0.781250,414.0,25.0,1.0,1.237437,29.955175,414.0,85.0,20.0,754.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
72770,0.937500,590.0,30.0,1.0,1.145644,32.643433,590.0,87.0,11.5,687.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
72771,0.645161,393.0,20.0,0.0,1.244342,33.187153,393.0,82.0,20.0,783.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [8]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

prePipe = Pipeline([
                     ('get_relavent_games',  SelectRowsTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectPosRowsTransformer('is_player', target=DATA_NAME)),
                    ])

In [9]:
preprocessor = PreprocessorBuilder(games, G_NAME, turns, T_NAME, prePipe).build()
n_games, n_turns, n_ratings = preprocessor.process(train, DATA_NAME)

In [10]:
n_turns

,.__mean,score__absolute_maximum,.__sum_values,.__median,.__root_mean_square,points__root_mean_square,score__maximum,TileType.N__sum_values,points__median,points__sum_values,...,W__absolute_maximum,T__sum_values,B__absolute_maximum,B__maximum,C__maximum,C__absolute_maximum,H__maximum,H__absolute_maximum,A__median,turn_type_Pass__median
game_id,,,,,,,,,,,,,,,,,,,,,
1,0.444444,429.0,12.0,0.0,0.816497,35.211530,429.0,78.0,24.0,764.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
3,0.481481,440.0,13.0,0.0,0.793492,34.230158,440.0,74.0,25.0,758.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,1.137931,478.0,33.0,1.0,1.790781,28.373737,478.0,73.0,16.0,597.0,...,1.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0
5,0.451613,427.0,14.0,0.0,0.762001,29.444316,427.0,74.0,21.0,752.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
6,0.600000,427.0,15.0,0.0,1.000000,37.758708,427.0,78.0,28.0,805.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,0.781250,414.0,25.0,1.0,1.237437,29.955175,414.0,85.0,20.0,754.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
72770,0.937500,590.0,30.0,1.0,1.145644,32.643433,590.0,87.0,11.5,687.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
72771,0.645161,393.0,20.0,0.0,1.244342,33.187153,393.0,82.0,20.0,783.0,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [11]:
n_data = n_games.merge(n_turns, left_index=True, right_index=True)

In [12]:
n_data.head()

,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,...,W__absolute_maximum,T__sum_values,B__absolute_maximum,B__maximum,C__maximum,C__absolute_maximum,H__maximum,H__absolute_maximum,A__median,turn_type_Pass__median
game_id,,,,,,,,,,,,,,,,,,,,,
1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,...,1.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0
5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
6,HivinD,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,...,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [13]:
featureTransformers = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                       ('hot', OneHotEncoderTransformer({'time_control_name': games['time_control_name'].unique(),
                                                          'game_end_reason':  games['game_end_reason'].unique(),
                                                          'lexicon':  games['lexicon'].unique(),
                                                          'rating_mode':  games['rating_mode'].unique(),
                                                          'bot_name': names}
                                                        )),
                       ('drops', NameDropperTransformer(['first', 'created_at', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'])),                                              
                      ]

In [14]:
reg_pipe = Pipeline(featureTransformers)
t_data = reg_pipe.transform(n_data)

In [15]:
grid = {
    'random_forest_1':
    {
    'estimator': RandomForestRegressor(random_state=0),
    'param_grid':
        {
        'max_depth': [40, 80, None],
        'max_features': ['sqrt', 'log2', 0.2],
        'min_samples_leaf': [50, 25, 10],
        },
    'scores': ['neg_root_mean_squared_error']
    },
    # 'random_forest_2':
    # {
    # 'estimator': RandomForestRegressor(random_state=0),
    # 'param_grid':
    #     {
    #     'max_depth': [2, 3],
    #     'max_features': ['sqrt', None]
    #     },
    # 'scores': ['neg_root_mean_squared_error']
    # },
}

In [16]:
my_searcher = Searcher(reg_pipe, grid, 5, redo_scores=False)
my_searcher.search(n_data, n_ratings)

In [ ]:
res = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher.get_results())

In [ ]:
res.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,max_depth,min_samples_leaf,max_features
model_name,,,,,,,,,,,,,,,
random_forest_1,8.374296,0.436494,1.092749,0.149992,-203.782272,-203.673123,-203.586736,-203.100572,-202.504091,-203.329359,0.473911,34,None,0.1,log2
random_forest_1,6.811836,0.603390,0.839980,0.141938,-203.782272,-203.673123,-203.586736,-203.100572,-202.504091,-203.329359,0.473911,34,20,0.1,log2
random_forest_1,8.287281,0.310422,0.919957,0.164383,-203.782272,-203.673123,-203.586736,-203.100572,-202.504091,-203.329359,0.473911,34,40,0.1,log2
random_forest_1,9.081824,0.250756,0.903303,0.061321,-196.172011,-196.496450,-195.643236,-195.531082,-195.556612,-195.879878,0.386694,31,40,0.05,log2
random_forest_1,11.814389,0.305191,1.329809,0.209110,-196.172011,-196.496450,-195.643236,-195.531082,-195.556612,-195.879878,0.386694,31,None,0.05,log2
random_forest_1,9.203409,0.302515,0.860297,0.065870,-196.172011,-196.496450,-195.643236,-195.531082,-195.556612,-195.879878,0.386694,31,20,0.05,log2
random_forest_1,9.695374,0.247609,0.662234,0.036129,-192.812581,-192.925373,-192.289501,-191.924982,-192.509355,-192.492358,0.361497,28,None,0.1,sqrt
random_forest_1,9.806407,0.906992,0.715164,0.088889,-192.812581,-192.925373,-192.289501,-191.924982,-192.509355,-192.492358,0.361497,28,40,0.1,sqrt
random_forest_1,35.403083,0.113275,1.343824,0.030156,-192.812581,-192.925373,-192.289501,-191.924982,-192.509355,-192.492358,0.361497,28,20,0.1,sqrt
